In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import random

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def load_set14_images(root_dir='Set14'):
    subfolders = ['image_SRF_2', 'image_SRF_3', 'image_SRF_4']
    images = []
    for subfolder in subfolders:
        subfolder_path = os.path.join(root_dir, subfolder)
        if not os.path.exists(subfolder_path):
            print(f"Subfolder {subfolder_path} does not exist.")
            continue
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".png") or filename.endswith(".jpg"):
                img_path = os.path.join(subfolder_path, filename)
                img = Image.open(img_path).convert('L')
                img = img.resize((128, 128), Image.BICUBIC)
                img_array = np.array(img) / 255.0 
                images.append(img_array)
    images = np.array(images)
    print(f"Loaded {len(images)} images from Set14 dataset.")
    return images

images = load_set14_images(root_dir='Set14')

In [ ]:
def plot_sample_image(images):
    plt.figure(figsize=(5, 5))
    plt.imshow(images[0], cmap='gray')
    plt.title('Sample Set14 Image')
    plt.axis('off')
    plt.show()

plot_sample_image(images)

In [ ]:
def extract_patches(images, patch_size=32, stride=16):
    patches = []
    for img in images:
        h, w = img.shape
        for i in range(0, h - patch_size + 1, stride):
            for j in range(0, w - patch_size + 1, stride):
                patch = img[i:i+patch_size, j:j+patch_size]
                patches.append(patch)
    patches = np.array(patches)
    print(f"Extracted {len(patches)} patches.")
    return patches
patches = extract_patches(images, patch_size=32, stride=16)

In [ ]:
def add_gaussian_noise(images, std_dev):
    noise = np.random.normal(0, std_dev, images.shape)
    noisy_images = images + noise
    noisy_images = np.clip(noisy_images, 0., 1.)
    return noisy_images

std_dev = 0.1
noisy_patches = add_gaussian_noise(patches, std_dev)

In [ ]:
def get_k_space_data(images):
    k_space = np.fft.fftshift(np.fft.fft2(images, axes=(-2, -1)))
    return k_space

k_space_noisy = get_k_space_data(noisy_patches)

In [ ]:
def prepare_data(k_space_noisy, patches):
    k_space_noisy_real = np.real(k_space_noisy)
    k_space_noisy_imag = np.imag(k_space_noisy)

    n = k_space_noisy_real.shape[1]
    X_real_flat = k_space_noisy_real.reshape(-1, n * n)
    X_imag_flat = k_space_noisy_imag.reshape(-1, n * n)
    X = np.concatenate([X_real_flat, X_imag_flat], axis=1)
    y = patches.reshape(-1, n, n, 1)
    return X, y

X, y = prepare_data(k_space_noisy, patches)

In [ ]:
def create_automap_model(n):
    inputs = Input(shape=(2 * n * n,))
    x = Dense(8 * n * n, activation='tanh')(inputs)
    x = Dense(8 * n * n, activation='tanh')(x)
    x = Dense(4 * n * n, activation='tanh')(x)
    x = Dense(n * n, activation='tanh')(x)
    x = Reshape((n, n, 1))(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs=inputs, outputs=x)
    return model

n = patches.shape[1]

In [ ]:
model = create_automap_model(n)
model.compile(optimizer=Adam(1e-4), loss='mean_squared_error')

print("AUTOMAP Model with Increased Capacity:")
model.summary()

history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1)

In [ ]:
def display_results(model, X, y, num_samples=5):
    indices = random.sample(range(len(X)), num_samples)
    for idx in indices:
        X_input = X[idx:idx+1]
        n = int(np.sqrt(X_input.shape[1] // 2))
        reconstructed = model.predict(X_input)[0, :, :, 0]
        target = y[idx, :, :, 0]
        k_space_noisy_real = X_input[:, :n*n].reshape(n, n)
        k_space_noisy_imag = X_input[:, n*n:].reshape(n, n)
        k_space_noisy_complex = k_space_noisy_real + 1j * k_space_noisy_imag
        noisy_image_complex = np.fft.ifft2(np.fft.ifftshift(k_space_noisy_complex))
        noisy_image = np.abs(noisy_image_complex)
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.title('Noisy Image')
        plt.imshow(noisy_image, cmap='gray')
        plt.axis('off')
        plt.subplot(1, 3, 2)
        plt.title('Reconstructed Image')
        plt.imshow(reconstructed, cmap='gray')
        plt.axis('off')
        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.imshow(target, cmap='gray')
        plt.axis('off')
        plt.show()

display_results(model, X, y, num_samples=5)
